In [8]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

train_data = np.loadtxt('train_data.txt', dtype=np.float32, delimiter='	',skiprows=1)
print(train_data.shape)
train_truth= np.loadtxt('train_truth.txt', dtype=np.float32, delimiter='	',skiprows=1)
test_data = np.loadtxt('test_data.txt', dtype=np.float32, delimiter='	',skiprows=1)

train_data = torch.from_numpy(train_data)
train_truth = torch.from_numpy(train_truth)
# x , y =(Variable(train_data),Variable(train_truth))

deal_dataset = TensorDataset(train_data, train_truth)
train_loader = DataLoader(dataset=deal_dataset, batch_size=100, shuffle=True, num_workers=2)
test_data = torch.from_numpy(test_data)
#test_predict = torch.from_numpy(test_predict)
testx = Variable(test_data)

class simpleNet(nn.Module):
    def __init__(self,n_input,n_hidden,n_output):
        super(simpleNet,self).__init__()
        self.hidden1 = nn.Linear(n_input,n_hidden)
        self.hidden2 = nn.Linear(n_hidden,n_hidden)
        self.predict = nn.Linear(n_hidden,n_output)
    def forward(self,input):
        out = self.hidden1(input)
        out = F.relu(out)
        out = self.hidden2(out)
        out = F.relu(out)
        out =self.predict(out)

        return out


if __name__ == '__main__':
    spNet = simpleNet(3, 4, 1)
    optimizer = torch.optim.Adam(spNet.parameters(),lr = 1e-4)
    loss_func = torch.nn.MSELoss()
    device = torch.device('cpu')
    losslist = []
    for epoch in range(150):
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            y = spNet(inputs)
            # print(y.shape)
            # print(labels.shape)
            loss = loss_func(y, labels.reshape([100,1]))
            losslist.append(loss)
            # print('{} epoch, loss:{}'.format(epoch, loss))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avgloss = sum(losslist)/ len(losslist)
        print('{} epoch, loss:{}'.format(epoch, avgloss))
    torch.save(spNet.state_dict(), 'car.pkl')
    
    # predict
    spNet.load_state_dict(torch.load('./car.pkl'))
    spNet.eval()
    print(testx)
    prediction = spNet(testx)
    print(prediction)
    list = [float(i) for i in prediction]
    # print(list)
    with open('./test_predicted.txt', 'w') as f:
        print('y',file=f)
        for element in list:
            print(element,file=f)


(10000, 3)
0 epoch, loss:0.1307951956987381
1 epoch, loss:0.12157420068979263
2 epoch, loss:0.11340754479169846
3 epoch, loss:0.10613296180963516
4 epoch, loss:0.09960836172103882
5 epoch, loss:0.09372327476739883
6 epoch, loss:0.08840572834014893
7 epoch, loss:0.08359929174184799
8 epoch, loss:0.07925600558519363
9 epoch, loss:0.07533030211925507
10 epoch, loss:0.07178030908107758
11 epoch, loss:0.06856580078601837
12 epoch, loss:0.06564940512180328
13 epoch, loss:0.0629945620894432
14 epoch, loss:0.06056772544980049
15 epoch, loss:0.058338262140750885
16 epoch, loss:0.056277964264154434
17 epoch, loss:0.054360032081604004
18 epoch, loss:0.052560362964868546
19 epoch, loss:0.050858598202466965
20 epoch, loss:0.049240317195653915
21 epoch, loss:0.04769754409790039
22 epoch, loss:0.046223729848861694
23 epoch, loss:0.044812608510255814
24 epoch, loss:0.043459370732307434
25 epoch, loss:0.042160142213106155
26 epoch, loss:0.04091209918260574
27 epoch, loss:0.03971296548843384
28 epoch, l